In [1]:
import json
import rppglib.train

In [2]:
class config:
    fps = 30
    ppg_low_freq = 0.5
    ppg_high_freq = 10
    batch_size = 32
    num_workers = 12
    train_dataset = 'mcd_rppg'
    test_datasets = ['mcd_rppg', 'MMPD', 'SCAMPS', 'UBFC_rPPG']
    num_folds = 5
    test_fold = 0
    valid_fold = 1
    model = 'OMIT'
    results_folder = 'results'

In [3]:
for valid_fold in range(1, 2):
    config.valid_fold = valid_fold
    config = rppglib.train.train_fold(config)
    cfg = {k:v for k, v in config.__dict__.items() if not k.startswith('__')}
    name = f'{config.results_folder}/{config.model}__{config.train_dataset}__{config.valid_fold}'
    with open(f'{name}.cfg', 'w') as fp:
        json.dump(cfg, fp)

Model OMIT loaded

Testing on mcd_rppg
Test files: 240
Testing: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:02<00:00,  3.04it/s]
Testing true PPGs: (240, 600)
Testing pred PPGs: (240, 600)
Mean true HRs: 83.291015625
Mean pred HRs: 82.27294921875
PPG MAE: 0.8006817698478699
HR MAE: 4.78271484375

Testing on MMPD
Test files: 114
Testing: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:04<00:00,  1.02s/it]
Testing true PPGs: (114, 600)
Testing pred PPGs: (114, 600)
Mean true HRs: 83.28022203947368
Mean pred HRs: 74.18277138157895
PPG MAE: 0.7720935344696045
HR MAE: 15.326891447368421

Testing on SCAMPS
Test files: 554
Testing: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 18/18 [00:06<00:00,  2.79it/s]
Testing true PPGs: (554, 600)
Tes

In [4]:
from glob import glob
import json
import numpy as np

def open_cfg(file):
    with open(file, 'rt') as f:
        data = json.loads(f.read())
    return data

cfgs = glob(f'results/{config.model}__{config.train_dataset}__*.cfg')
cfgs = [open_cfg(file) for file in cfgs]

for ds in config.test_datasets:
    vals_ppg = [cfg['test_results'][f'test__{ds}__ppg'] for cfg in cfgs]
    vals_hr = [cfg['test_results'][f'test__{ds}__hr'] for cfg in cfgs]

    print()
    print(f'Test results on "{ds}" dataset:')
    print(f'HR  MAE: {np.mean(vals_hr):.2f}+/-{np.std(vals_hr):.3f}')
    print(f'PPG MAE: {np.mean(vals_ppg):.2f}+/-{np.std(vals_ppg):.3f}')


Test results on "mcd_rppg" dataset:
HR  MAE: 4.78+/-0.000
PPG MAE: 0.80+/-0.000

Test results on "MMPD" dataset:
HR  MAE: 15.33+/-0.000
PPG MAE: 0.77+/-0.000

Test results on "SCAMPS" dataset:
HR  MAE: 16.27+/-0.000
PPG MAE: 0.86+/-0.000

Test results on "UBFC_rPPG" dataset:
HR  MAE: 1.95+/-0.000
PPG MAE: 0.84+/-0.000
